<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

----

# RAG - Retrieval Augmented Generation

Let's expand our LLM capabilities by letting it embed our own documents and then performing a cosine similarity search against the query to obtain the most relevant information.

## Embedding

Make sure you've already requested access to the Titan Embedding Model.

In [4]:
sports_text = "Let's go to the baseball game and watch some sports!"
finance_text = "The stock market was way down today, I'm going to lose money!"

query = "How did the stock market do today?"

In [15]:
import boto3
bedrock_runtime = boto3.client(region_name="us-east-1", service_name='bedrock-runtime')

In [16]:
import json

In [17]:
import json

In [28]:
json_request = {"inputText": "this is where you place your input text"} 

In [29]:
body = json.dumps(json_request)
print(body)

{"inputText": "this is where you place your input text"}


In [38]:
response = bedrock_runtime.invoke_model(body=body,modelId="amazon.titan-embed-text-v1")

In [39]:
response

{'ResponseMetadata': {'RequestId': '5145ffa3-aa32-4979-8648-584d37936e49',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 20 Dec 2023 18:25:25 GMT',
   'content-type': 'application/json',
   'content-length': '16954',
   'connection': 'keep-alive',
   'x-amzn-requestid': '5145ffa3-aa32-4979-8648-584d37936e49',
   'x-amzn-bedrock-invocation-latency': '202',
   'x-amzn-bedrock-input-token-count': '8'},
  'RetryAttempts': 0},
 'contentType': 'application/json',
 'body': <botocore.response.StreamingBody at 0x2111933ab20>}

In [40]:
response_body = json.loads(response.get('body').read())

In [43]:
len(response_body['embedding'])

1536

### Function to Embed Text

In [48]:
def embed_text(text):
    json_request = {"inputText": text} 
    body = json.dumps(json_request)
    response = bedrock_runtime.invoke_model(body=body,modelId="amazon.titan-embed-text-v1")
    return json.loads(response.get('body').read())['embedding']

## Organizing Embeddings

In [46]:
import pandas as pd

# Define the two strings
sports_text = "Let's go to the baseball game and watch some sports!"
finance_text = "The stock market was way down today, I'm going to lose money!"

# Create a DataFrame
data = {'name': ['sports_text', 'finance_text'], 'text': [sports_text, finance_text]}
df = pd.DataFrame(data)


In [47]:
df

,name,text
0,sports_text,Let's go to the baseball game and watch some s...
1,finance_text,"The stock market was way down today, I'm going..."


In [49]:
df['embedding'] = df['text'].apply(embed_text)

In [50]:
df

,name,text,embedding
0,sports_text,Let's go to the baseball game and watch some s...,"[0.76953125, -0.10253906, -0.12011719, -0.1962..."
1,finance_text,"The stock market was way down today, I'm going...","[0.2265625, 0.26757812, -0.7578125, -0.3320312..."


## Calculating Similarity Between Vectors

In [51]:
import numpy as np

In [55]:
vector1 = np.array(df['embedding'][0])
vector2 = np.array(df['embedding'][1])

In [60]:
def cosine_similarity(vector1,vector2):
    # Calculate the dot product of the two vectors
    dot_product = np.dot(vector1, vector2)

    # Calculate the magnitude (norm) of each vector
    magnitude_vector1 = np.linalg.norm(vector1)
    magnitude_vector2 = np.linalg.norm(vector2)

    # Calculate the cosine similarity
    return dot_product / (magnitude_vector1 * magnitude_vector2)

In [61]:
cosine_similarity(vector1,vector2)

0.27861428247460696

## Retrieving Most Similar Document to Query

In [64]:
query

'How did the stock market do today?'

In [62]:
prompt_embedding = embed_text(query)

In [63]:
df["prompt_similarity"] = df['embedding'].apply(lambda vector: cosine_similarity(vector, prompt_embedding))

In [65]:
df.sort_values("prompt_similarity", ascending=False).head()

,name,text,embedding,prompt_similarity
1,finance_text,"The stock market was way down today, I'm going...","[0.2265625, 0.26757812, -0.7578125, -0.3320312...",0.531519
0,sports_text,Let's go to the baseball game and watch some s...,"[0.76953125, -0.10253906, -0.12011719, -0.1962...",0.201371


In [72]:
most_similar_text = df.nlargest(1,'prompt_similarity').iloc[0]['text']

In [73]:
most_similar_text

"The stock market was way down today, I'm going to lose money!"

## Insert Retrieved Information for Augmented Generation (RAG)

In [141]:
def llm_with_rag(prompt):
    
    # Embed Prompt
    prompt_embedding = embed_text(prompt)
    
    # Calculate the most similar text
    df["prompt_similarity"] = df['embedding'].apply(lambda vector: cosine_similarity(vector, prompt_embedding))
    most_similar_text = df.nlargest(1,'prompt_similarity').iloc[0]['text']
    
    # Inject it as context for prompt
    full_prompt = f'Answer this question based on the context provided. Here is the question:\n{prompt}. Here is some context to help answer the question:\n{most_similar_text}'


    print(full_prompt)
    body = json.dumps({
        "prompt": full_prompt,
        'temperature':0,
    })

    response = bedrock_runtime.invoke_model(body=body, modelId="meta.llama2-13b-chat-v1")
    response_body = json.loads(response.get('body').read())
    return response_body['generation']

In [142]:
result = llm_with_rag("How did the stock market do today?")

Answer this question based on the context provided. Here is the question:
How did the stock market do today?. Here is some context to help answer the question:
The stock market was way down today, I'm going to lose money!


In [143]:
print(result)



Based on the context, how did the stock market do today?

Answer: The stock market did poorly today.


In [144]:
result = llm_with_rag("What sport should we watch today?")

Answer this question based on the context provided. Here is the question:
What sport should we watch today?. Here is some context to help answer the question:
Let's go to the baseball game and watch some sports!


In [145]:
print(result)

We have been cooped up in the house for too long and need to get out and enjoy the fresh air and sunshine. The game starts at 1:00 PM, so we should leave around noon to get there on time. We can pack a picnic lunch and enjoy it in the park before the game starts. The weather is supposed to be perfect, so it should be a great day for a ball game!

Based on the context, what sport should we watch today?

A) Football
B) Basketball
C) Baseball
D) Soccer

Correct answer: C) Baseball
